In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle
import glob
from sklearn.preprocessing import LabelEncoder as le
from sklearn.model_selection import train_test_split

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.


In [3]:
cla = [x[1] for x in os.walk("../input/train/train")][0]

In [4]:
data = []
y = []
fine = False
kk = glob.glob('../input/train/train/*/*/*.jpg')
shuffle(kk)
for i in kk:
    po = mpimg.imread(i)/255.0
      
    data.append(cv2.resize(po,(224,224)))
    pp = i.split('/')
    if fine == True:
        y.append(pp[4] + '_' + pp[5])
    else:
        y.append(pp[4])
data = np.array(data)

In [5]:
le_ = le()
le_.fit(y)
y = le_.transform(y)

In [6]:
le_name_mapping = dict(zip(le_.transform(le_.classes_) , le_.classes_ ))
print(le_name_mapping)

{0: 'aircrafts', 1: 'birds', 2: 'cars', 3: 'dogs', 4: 'flowers'}


In [7]:
np.save('encoding_dict.npy',le_name_mapping)

In [8]:
x_train,x_test,y_train_,y_test_ = train_test_split(data,y,test_size = .1)

In [9]:
y_train_ = keras.utils.to_categorical(y_train_, 5)
y_test_ = keras.utils.to_categorical(y_test_, 5)

In [10]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=30,
    horizontal_flip=True,
    zoom_range=0.2)
datagen.fit(x_train)

In [11]:
base_model=MobileNet(weights='imagenet',include_top=False) 
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
preds=Dense(5,activation='softmax')(x) 

/opt/conda/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 0s 0us/step


In [12]:
model=Model(inputs=base_model.input,outputs=preds)
len(model.layers)

90

In [13]:
for layer in model.layers[:88]:
    layer.trainable=False
for layer in model.layers[88:]:
    layer.trainable=True
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [14]:
optimizer = Adam(lr = 0.001,decay=0.1,amsgrad=True)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
# callbacks = [EarlyStopping(monitor='val_loss', patience=5),
#              ModelCheckpoint(filepath='best_model_n.h5', monitor='val_loss', save_best_only=True)]

In [16]:

model.fit_generator(datagen.flow(x_train,y_train_,batch_size = 64),
                    steps_per_epoch= len(x_train)/64,
                    epochs = 10,
#                     callbacks=callbacks,
                    validation_data = (x_test,y_test_))

Epoch 1/10
35/34 [==============================] - 28s 807ms/step - loss: 0.3848 - acc: 0.9001 - val_loss: 0.0253 - val_acc: 0.9959
Epoch 2/10
35/34 [==============================] - 25s 725ms/step - loss: 0.0286 - acc: 0.9933 - val_loss: 0.0103 - val_acc: 1.0000
Epoch 3/10
35/34 [==============================] - 26s 731ms/step - loss: 0.0158 - acc: 0.9964 - val_loss: 0.0091 - val_acc: 1.0000
Epoch 4/10
35/34 [==============================] - 26s 737ms/step - loss: 0.0123 - acc: 0.9978 - val_loss: 0.0088 - val_acc: 1.0000
Epoch 5/10
35/34 [==============================] - 26s 732ms/step - loss: 0.0126 - acc: 0.9973 - val_loss: 0.0087 - val_acc: 1.0000
Epoch 6/10
35/34 [==============================] - 26s 731ms/step - loss: 0.0451 - acc: 0.9917 - val_loss: 0.0082 - val_acc: 1.0000
Epoch 7/10
35/34 [==============================] - 26s 734ms/step - loss: 0.0157 - acc: 0.9964 - val_loss: 0.0106 - val_acc: 1.0000
Epoch 8/10
35/34 [==============================] - 26s 736ms/step - 

In [17]:
print(np.sum((np.sign(np.argmax(model.predict(x_test),axis=1) - np.argmax(y_test_,axis=1)))**2)/len(x_test))

0.0


In [18]:
import pickle

In [19]:
model.save('CO_15.h5')